### Descarga de capas de mapa de cobertura 4G del Instituto Federal de Telecomunicaciones

In [1]:
import requests
import geopandas as gpd
import pandas as pd
from bs4 import BeautifulSoup
import os
import json

In [2]:
#Crear directorio para guardar los archivos
os.makedirs("geopaquetes", exist_ok=True)
#Cambiar al directorio
os.chdir("geopaquetes")

Paso 1: Se obtiene el código html de la página en donde se encuentra el mapa.

In [3]:
url="https://felt.com/map/Mapa-interactivo-de-cobertura-4G-IFT-nkwEcoI4S9BSa7vgFt6uctD?loc=20.0624,-98.7601,11.6z"

In [4]:
#Obtener el mapa
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# obtener div id="felt-data"
div = soup.find("div", {"id": "felt-data"})
#Eliminar textos
div = str(div).replace('<div id="felt-data" style="display: none">', '')
div = div.replace('\n  </div>', '')
div=div.replace('\n','')

In [6]:
#Transformar a json
data = json.loads(div)

In [7]:
data.keys()

dict_keys(['mapColorPalette', 'loadedAt', 'maptiler_key', 'canCreateMapsSomewhere', 'layerProcessingEmailSubscriptions', 'layerAttrConstraints', 'settings', 'ghostPresences', 'mapTeamId', 'selectedBackgroundId', 'embedConfig', 'isTrainingKarta', 'maxTileURLLength', 'sources', 'urls', 'showBasemapLabels', 'mapBackgrounds', 'felt_version', 'featureFlags', 'isAdminView', 'defaultZoom', 'validGeoDataExtensions', 'currentTeamBannerAcknowledgement', 'updatedAtUnixMs', 'mapLinks', 'resyncTimeoutMs', 'mapbox_api_token', 'mapProject', 'workspaceBilling', 'isVirtualKarta', 'pipeline', 'checkerboardTiles', 'satelliteMode', 'current_user', 'createMapParams', 'accessControl2', 'elements', 'kartaChannel', 'teams', 'defaultControlsMode', 'defaultCoordinates', 'onboardingNeeded', 'mapDescription', 'mapImages', 'folderId', 'authors', 'individualPermissions', 'mapId', 'allowedFeatures', 'isOwner', 'validMapImageMimeTypes', 'editableByCurrentSession', 'widgets', 'socketTokenTTLSeconds', 'customViewport',

In [8]:
data["layerGroups"][0].keys()

dict_keys(['id', 'name', 'visible', 'description', 'created_at', 'layers', 'created_by', 'modified_at', 'user_id', 'max_zoom', 'subtitle', 'z_order', 'index_json_url', 'hideFromLegend', 'isCollapsed', 'errorMessage', 'thumbnailUrl', 'progress_percent', 'visibilityInteraction', 'created_at_unix_time_ms', 'duplicatedFromId', 'errorType', 'renderAsLayer'])

Paso 2: Se obtienen los nombres y urls de la información de cada capa del mapa

In [9]:
#Obtener nombres y urls
nombres=[]
urls=[]
for i in range(len(data["layerGroups"])):
    nombres.append(data["layerGroups"][i]["layers"][0]["normalized"]["layername"])
    urls.append(data["layerGroups"][i]["layers"][0]["index_json_url"])

In [10]:
#Crear tabla
tabla = pd.DataFrame({"nombre":nombres, "url":urls})
#Eliminar último elemento
tabla = tabla[:-1]
#Renombrar último registro como "Movistar"
tabla["nombre"][3]="Movistar"
tabla["nombre"][0]="ATT"
tabla

,nombre,url
0,ATT,https://data-pipeline.felt.com/upload/6b433aae...
1,Telcel,https://data-pipeline.felt.com/upload/b03fd7d1...
2,ALTAN Redes,https://data-pipeline.felt.com/upload/c80325bd...
3,Movistar,https://data-pipeline.felt.com/upload/6b433aae...


Paso 3: Se obtienen las urls de los datos de cada capa y se descargan los archivos

In [11]:
#Obtener las urls de la data de los mapas
urls_data=[]
gdf=gpd.GeoDataFrame()
for i in range(len(tabla)):
    url = tabla["url"][i]
    response = requests.get(url)
    data= response.json()
    urls_data.append(data["datasets"][0]["data_url"])
    #descargar el geopackage de cada url con el nombre de la capa
    nombre_archivo = tabla["nombre"][i]
    write_path = f"{nombre_archivo}.gpkg"
    with open(write_path, 'wb') as f:
        f.write(requests.get(data["datasets"][0]["data_url"]).content)

Paso 4: Se cargan los archivos geopackage y se concatenan en un solo archivo

In [12]:
gdfs = []
#Cargar datos
for i in range(len(tabla)):
    nombre_archivo = tabla['nombre'][i]
    gdf = gpd.read_file(f"{nombre_archivo}.gpkg")
    #Incluir el nombre de la capa
    gdf["nombre"] = tabla["nombre"][i]
    gdfs.append(gdf)
    
# Concatenar los geodataframes
gdf_final = pd.concat(gdfs, ignore_index=True)

In [13]:
gdf_final

,felt:feature,felt:has_geometry,Calidad,geometry,nombre
0,1,True,Buena,"MULTIPOLYGON (((-117.12184 32.52598, -117.1241...",ATT
1,2,True,Buena,"MULTIPOLYGON (((-117.12181 32.52263, -117.1241...",ATT
2,3,True,Excelente,"MULTIPOLYGON (((-117.12165 32.50254, -117.1239...",ATT
3,4,True,Excelente,"MULTIPOLYGON (((-117.12162 32.49919, -117.1239...",ATT
4,5,True,Excelente,"MULTIPOLYGON (((-117.15130 32.35282, -117.1535...",ATT
...,...,...,...,...,...
1285015,250821,True,Regular,"MULTIPOLYGON (((-97.33838 18.69422, -97.34039 ...",Movistar
1285016,250822,True,Regular,"MULTIPOLYGON (((-97.33561 18.69221, -97.33762 ...",Movistar
1285017,250823,True,Regular,"MULTIPOLYGON (((-97.33465 18.69887, -97.33665 ...",Movistar
1285018,250824,True,Regular,"MULTIPOLYGON (((-97.33332 18.68687, -97.33533 ...",Movistar


In [14]:
type(gdf_final)

geopandas.geodataframe.GeoDataFrame

Paso 5: Se guarda el archivo final en formato geopackage

In [ ]:
#Salvar como geopackage
gdf_final.to_file("cobertura_4g.gpkg", driver="GPKG",crs="EPSG:4326")